# Preprocesamiento de texto

Cuando los datos de entrada de nuestro algoritmo de *machine learning* son textos, tenemos un problema similar al que sucedía cuando los datos de entrada eran variables discretas: los algoritmos de *machine learning* funcionan a partir de operaciones matemáticas que no pueden operar con textos. No existe defidida ninguna operación matemática que puede combinar las palabras _"hola"_ y _"adiós"_. Por tanto, para poder emplear textos definidos en lenguaje natural, independientemente del idioma empleado, necesitamos transformar esos textos en vectores numéricos que los representen.

La técnica más conocida para hacer esta transformación se denomina ***bag of words***. Veamos cómo funciona con un ejemplo. Supongamos que tenemos el siguiente texto:

> "*El miedo es el camino hacia el lado oscuro, el miedo lleva a la ira, la ira lleva al odio, el odio lleva al sufrimiento, el sufrimiento al lado oscuro.*"

El primer paso que debemos realizar es el que conocemos como ***tokenizacion***, que consiste en trasformar el texto anterior en un array de palabras. Es decir, vamos a separar cada una de las palabras que conforman la frase anterior empleando como separador los espacios y signos de puntuación. Por tanto, obtendríamos la siguiente lista de *tokens*:


`['El', 'miedo', 'es', 'el', 'camino', 'hacia', 'el', 'lado', 'oscuro', 'el', 'miedo', 'lleva', 'a', 'la', 'ira', 'la', 'ira', 'lleva', 'al', 'odio', 'el', 'odio', 'lleva', 'al', 'sufrimiento', 'el', 'sufrimiento', 'al', 'lado', 'oscuro']`

Ahora vamos a homogeneizar nuestro texto transformando todas las palabras a minúsculas:

`['el', 'miedo', 'es', 'el', 'camino', 'hacia', 'el', 'lado', 'oscuro', 'el', 'miedo', 'lleva', 'a', 'la', 'ira', 'la', 'ira', 'lleva', 'al', 'odio', 'el', 'odio', 'lleva', 'al', 'sufrimiento', 'el', 'sufrimiento', 'al', 'lado', 'oscuro']`

A partir de la lista anterior podemos construir un diccionario que contiene todas las palabras que están definidas en nuestro vocabulario. Entendemos como "nuestro vocabulario" a las palabras que aparecen en los textos que estamos analizando. El algoritmo de *machine learning* no necesita conocer si esa palabra pertenece o no al Diccionario de la Real Academia de la Lengua Española (o su equivalente en otros idiomas). Así pues, analizando los *tokens* anteriores construiremos el siguiente diccionario:

`['el', 'miedo', 'es', 'camino', 'hacia', 'lado', 'oscuro', 'lleva', 'a', 'la', 'ira', 'odio', 'sufrimiento']`

Por último, transformar el texto original en un vector numérico de tal forma que las posiciones del vector representan las posiciones de las palabras del diccionario y los valores del vector representa el número de apariciones de la palabra del diccionario en el texto analizado. Nuestro texto quedaría, por tanto, definido por el siguiente vector:

`[6, 2, 1, 1, 1, 2, 2, 3, 1, 2, 2, 2, 2]`

Analizándolo vemos que la palabra *'el'* se repite 6 veces, la palabra *'miedo'* 2, la palabra *'es'* 1, y así sucesivamente.

In [ ]:
import sklearn
import numpy as np

*sklearn* nos da soporte para transformar textos en su presentanción mediante *bag of words*. Para ello emplearemos el objeto [feature_extraction.text.CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) del siguiente modo.

Primero, importamos el módulo y definimos nuestro objeto:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

A continuación, aplicamos la transformación a nuestro texto de ejemplo. En este caso, *CountVectorizer* está esperando una secuencia de textos, a la que se denomina ***corpus***, por lo que debemos declarar nuestro texto dentro de un array de 1 elemento.

In [ ]:
corpus = [
    "El miedo es el camino hacia el lado oscuro, el miedo lleva a la ira, la ira lleva al odio, el odio lleva al sufrimiento, el sufrimiento al lado oscuro."
]

X = count_vectorizer.fit_transform(corpus)

print(X.toarray())


Podemos analizar qué palabra corresponden a cada posición de este array:

In [ ]:
print(count_vectorizer.get_feature_names())

Llegados a este punto es importante resaltar que *sklearn* almacena los vectores en una matriz dispersa que proporciona *NumPy*. Esto se debe a que, cuando trabajamos con textos, lo normal es que el *corpus* disponga de cientos o miles textos (llamados ***documentos***) sobre los cuales se construye el *diccionario*. Como es lógico, no todos los *documentos* contienen todas las plabras del *diccionario*, por lo que, habitualmente, los vectores de *bag of words* están repletos de valores 0. Esta es una información irrelevante que desperdicia gran cantidad de espacio en memoria, por lo que se alamcena utilizando otro tipo de estrucutras de datos que sólo guarda qué palabras pertenecen a cada documento.

Si analizamos con detalle el vector del texto de ejemplo vemos que las palabras con mayor número de repeticiones son *'el'*, con 6 repeticiones, *'al'*, con 3 repeticiones y *'lleva'* con 3 repeticiones. Esto es un gran problema, puesto que las 2 primeras no aportan ningún significado semántico al texto y provocará que nuestro algoritmo de *machine learning* no sea capaz de extraer conocimiento de *corpus* de documentos.

Para resolver este problema se filtran estas palabras del *corpus* antes de construir el diccionario. A estas palabras, que en realidad son todos los artículos, preposiciones, etc., se las conoce como ***stop words***, y existen listas en diferentes idiomas para realizar este filtrado. Veamos cómo.

Cuando trabajamos con texto es extremadamente útil conocer la librería NLTK ([thttps://www.nltk.org/](https://www.nltk.org/)), ya que incorpora infinidad de herramientas para la manipulación de textos en lenguaje natural. Entre otras funcionalidades, incorpora una lista de *stop words* en diferentes idiomas. Carguemos las *stop words* en español:

In [ ]:
import nltk
nltk.download("popular") # required to download the stopwords lists

from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')

Las visualizamos:

In [ ]:
print(spanish_stopwords)

Ahora, podemos crearnos una instancia de nuestro *CountVectorizer* que incluya esta lista de *stop words* utilizando el parámetro *stop_words* de su constructor:

In [ ]:
count_vectorizer = CountVectorizer(stop_words = spanish_stopwords)

Y repetimos el proceso anterior:

In [ ]:
corpus = [
    "El miedo es el camino hacia el lado oscuro, el miedo lleva a la ira, la ira lleva al odio, el odio lleva al sufrimiento, el sufrimiento al lado oscuro."
]

X = count_vectorizer.fit_transform(corpus)

print(X.toarray())

Si analizamos las palabras del diccionario vemos que no hay ni rastro de las *stop words*:

In [ ]:
print(count_vectorizer.get_feature_names())

## tf-idf

Aunque llegados a este punto ya tenemos una buena representación vectorial de nuestros textos, aún existe un problema: la representación creada no está normalizada. Esta no-normalización plantea básicamente dos problemas:

- A nivel de *documento* (las filas de nuestra matriz de datos) cada uno lleva una escala completamente libre y hace que sea imposible compararlos entre si. Un texto más largo tendrá contadores con valores más grandes que un texto más corto. En un ejemplo llevado al extremo podemos comparar un tweet con la noticia que enlaza ese tweet. Ambos documentos versarán sobre el mismo tema, pero no pueden compararse debido a la volumetría de ambos.

- A nivel de *palabras* es complicado comparar cuáles son más relevante y cuáles menos para un *corpus* concreto. Ya hemos eliminado las *stop words*, pero, en función del sesgo de nuestro *corpus* existen palabras que no aportan demasiada información y, por lo tanto, su incidencia en nuestro algoritmo de *machine learning* debería ser menor. Por ejemplo, imaginemos que tenemos un *corpus* de documentos hablando únicamente de los equipos de la Liga de Fútbol Profesional. En este corpus la palabra *'fútbol'* es completamente irrelevante, ya que todos los documentos hablan de ella. Por contra, palabras como *'lesión'* o *'fichaje'* son muy relevante porque permiten subclasificar los documentos. Sin embargo, si nuestro *corpus* está formado por noticias de todo tipo, la palabra *'fútbol'* es muy relevante ya que identifica un tipo de noticias.

Para resolver este problema se emplea una normalización denominada **tf-idf** (*term-frecuency times inverse document-frecuency*). Ésta viene definida por la siguiente ecuación:

$\textrm{tf-idf}(t, d) = tf(t, d) \times idf(t)$

siendo $tf(t, d)$ el número de veces que aparece el término (palabra) $t$ en el documento $d$ y definiéndose $idf(t)$ como:

$idf(t) = log \frac{1 + n}{1 + df(t)} + 1$

siendo $n$ el número de documentos de nuestro *corpus* y $df(t)$ el número de documentos en los que aparece el término $t$.

Posteriormente, los vectores son normalizados a nivel de documento (el modulo del vector de cada documento vale 1).

Analizando estas ecuaciones *tf-idf* observamos que, aquellas palabras que tengan menos frecuencia de aparición serán más relevante que aquellas que aparezcan en más documentos.

Esta transformación puede realizarse mediante el objeto [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer).

Para ver su funcionamiento, vamos a construir un *corpus* con varios documentos:

In [ ]:
corpus = [
    "Este es el primer documento.",
    "Este documento es el segundo documento.",
    "Y este es el tercero",
    "¿Es este el primer documento? No."
]

Aplicando el proceso anterior obtenemos los siguientes vectores:

In [ ]:
X = count_vectorizer.fit_transform(corpus)

print(X.toarray())
count_vectorizer.get_feature_names()

Veamos como aplicar ahora *tf-idf*. Cargamos el módulo y creamos el objeto:

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

Realizamos la transformación:

In [ ]:
counts = X.toarray()
X_transformed = tfidf_transformer.fit_transform(counts)

Analizamos el resultado:

In [ ]:
print(X_transformed.toarray())

## Stemming

Hasta ahora lo que tenemos es que cada uno de los términos o palabras de los documentos se transforman en una variable del conjunto de datos. Esto da lugar a una alta dimensionalidad del problema.

Una posible solución pasa por aplicar un filtrado previo conocido como __Stemming__, que no es otra cosa que truncar las palabras y quedarnos únicamente con su raíz. Por ejemplo, las palabras _chocolate_ y _chocolatería_ son dos términos distintos, por lo que cada una de ellas se convertirían en una variable. Sin embargo, si aplicamos __stemming__ ambos términos se transformarían en `chocolate`, por lo que se contarían como dos ocurrencias del mismo término.

Este filtrado tiene sentido _a priori_, pues al quedarnos con la raíz de cada palabra nos estamos quedando realmente con el _significado global_.

Un ejemplo:

In [ ]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

english_stopwords = stopwords.words('english')

ps = PorterStemmer()

frase = """The man who passes the sentence should swing the sword. 
If you would take a man's life, you owe it to him to look into his eyes and hear his final words. 
And if you cannot bear to do that, then perhaps the man does not deserve to die"""

palabras = word_tokenize(frase)
palabras = [p for p in palabras if not p in set(english_stopwords)]
for p in palabras:
    print(p, " : ", ps.stem(p))

## Lematización

La __lematización__ es el proceso de agrupar las diferentes formas inflexionadas de una palabra para que puedan ser analizadas como un solo elemento. La lematización es similar al _stemming_, pero aporta contexto a las palabras. Por lo tanto, vincula las palabras que tienen un significado similar.

Generalmente, la lematización es preferible al _stemming_ porque la lematización hace un análisis morfológico de las palabras.

Utilizando la frase anterior como ejemplo:

In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

for p in palabras:
    print ("{0:20}{1:20}".format(p,wordnet_lemmatizer.lemmatize(p)))

Se puede observar que no se ha realizado ninguna sustitución de los términos por su _lema_. Esto se debe a que es necesario especificar el rol de cada término dentro de la oración, es decir, si es un verbo, sustantivo, adjetivo, etc.

In [ ]:
for p in palabras:
    print ("{0:20}{1:20}".format(p,wordnet_lemmatizer.lemmatize(p, pos='v')))

---

*Este documento ha sido desarrollado por **Fernando Ortega** y **Raúl Lara**. Dpto. Sistemas Informáticos, ETSI de Sistemas Informáticos, Universidad Politécnica de Madrid.*

*Última actualización: Marzo de 2020*

<img src="https://drive.google.com/uc?export=view&id=1QuQDHyH_yrRbNt6sGzoZ8YcvFGEGlnWZ" alt="CC BY-NC">